# KAKENの審査区分マスタをローカルのMariaDBに保存するプログラム

## 事前準備

- KAKENマスタデータは、git のリポジトリで管理されており、最新のデータを利用可能。
- https://bitbucket.org/niijp/grants_masterxml_kaken/ からリポジトリを pull して、ローカルの ./grants_masterxml_kaken フォルダに同期しておく。
- ローカルで MariaDB を動かしておく。
- MariaDB のユーザ名、パスワード、データベース名は、config.ipynbを使って ./config.ini として保存しておく。

## ここから本編

bitbucketから読み込んだマスタのXMLファイルをelementTreeに変換

In [ ]:
from lxml import etree
import pandas as pd
import numpy as np

In [ ]:
tree = etree.parse('grants_masterxml_kaken/review_section_master_kakenhi.xml')

review_sectionlist = []
for review_section_table in tree.iterfind("review_section_table"):
    review_section_table_type = review_section_table.get("type")
    review_section_table_start_date = review_section_table.get("start_date")
    review_section_table_end_date = review_section_table.get("end_date")
    
    for review_section in review_section_table.iterfind("review_section"):
        review_section_path = review_section.get("path")
        review_section_niicode = review_section.find("code[@type='nii']").text
        review_section_name = review_section.find("name[@lang='ja']").text
        row = [
            review_section_table_type,
            review_section_table_start_date,
            review_section_table_end_date,
            review_section_path,
            review_section_niicode,
            review_section_name,
            1,
        ]
        review_sectionlist.append(row)

        for review_section in review_section.iterfind("review_section"):
            review_section_path = review_section.get("path")
            review_section_niicode = review_section.find("code[@type='nii']").text
            review_section_name = review_section.find("name[@lang='ja']").text
            row = [
                review_section_table_type,
                review_section_table_start_date,
                review_section_table_end_date,
                review_section_path,
                review_section_niicode,
                review_section_name,
                2,
            ]
            review_sectionlist.append(row)
                
            for review_section in review_section.iterfind("review_section"):
                review_section_path = review_section.get("path")
                review_section_niicode = review_section.find("code[@type='nii']").text
                review_section_name = review_section.find("name[@lang='ja']").text
                row = [
                    review_section_table_type,
                    review_section_table_start_date,
                    review_section_table_end_date,
                    review_section_path,
                    review_section_niicode,
                    review_section_name,
                    3,
                ]
                review_sectionlist.append(row)

df = pd.DataFrame(review_sectionlist)
df.columns = [
    'review_section_table_type',
    'review_section_table_start_date',
    'review_section_table_end_date',
    'review_section_path',
    'review_section_niicode',
    'review_section_name',
    'layer',
]

In [ ]:
df['review_section_table_start_date'] = pd.to_datetime(df['review_section_table_start_date'])
df['review_section_table_end_date'] = pd.to_datetime(df['review_section_table_end_date'])
df = df.astype({
    'review_section_niicode': int,
    'layer': int,
})

In [ ]:
df.info()

ローカルのmariaDBに関する設定ファイルを読み込みして、データベースにテーブル構造とデータの中身を書き込む

In [ ]:
import configparser

config = configparser.ConfigParser()
config.read('../settings/config.ini')
username = config['mariadb']['username']
password = config['mariadb']['password']
database = config['mariadb']['database']
url = 'mysql+pymysql://' + username + ':' + password + '@localhost:3306/' + database + '?charset=utf8'

from sqlalchemy import create_engine
from sqlalchemy.types import String, Integer, Date

engine = create_engine(url, echo=True)
df.to_sql('kaken_master_review_section', engine, if_exists='replace',
          dtype={
              'review_section_table_type' : String(256),
              'review_section_table_start_date': Date,
              'review_section_table_end_date': Date,
              'review_section_path': String(256),
              'review_section_niicode': Integer ,
              'review_section_name': String(256),
              'layer': Integer,
          })

## おしまい

データがコミットされていれば終了。HeidiSQLなどで、上記ドロップ済みデータフレームの件数が登録されているか確認する。